* Imports

In [1]:
from pytube import extract
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel,RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

d:\Programming\GEN-AI\Campus x Gen AI with langchain\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

Step-1a Indexing (Document Ingestion)

In [3]:
url = 'https://www.youtube.com/watch?v=hmtuvNfytjM&t=417s'
video_id = extract.video_id(url)

In [4]:
ytt_api=YouTubeTranscriptApi()
fetched_transcript = ytt_api.fetch(video_id,languages=['en','hi'])
transcript_list = fetched_transcript.to_raw_data()
transcript = ' '.join(snippet.text for snippet in fetched_transcript)
print(transcript)

This is like a crazy amount of power for 
one piece of technology and it's happened   to us so fast. You just launched GPT-5. A kid 
born today will never be smarter than AI. How   do we figure out what's real and what's not 
real? We haven't put a sex bot avatar in ChatGPT   yet. Super intelligence. What does that 
actually mean? This thing is remarkable. I'm about to interview Sam Alman, the CEO 
of Open AI. Open AI. Open AI. Reshaping   industries. Dude's a straightup tech lord. Let's 
be honest. Right now, they're trying to build a   super intelligence that could far exceed humans 
in almost every field. And they just released   their most powerful model yet. Just a couple years 
ago, that would have sounded like science fiction.   Not anymore. In fact, they're not alone. We are 
in the middle of the highest stakes global race   any of us have ever seen. Hundreds of billions of 
dollars and an unbelievable amount of human worth.   This is a profound moment. Most people never 
live 

Step 1b Indexing (text Splitting)

In [5]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks=splitter.create_documents([transcript])

In [6]:
len(chunks)

78

In [7]:
print(chunks[67])

page_content='parent telling me I was doing a good job."   Like I can get why this was bad for other people's 
mental health, but this was great for my mental   health. Like I didn't realize how much I needed 
this. It encouraged me to do this. It encouraged   me to make this change in my life. Like it's 
not all bad for chatbt to it turns out like be   encouraging of you. Now the way we were doing 
it was bad, but turn it like something in that   direction might have some value in it. How we do 
it, we we show the model examples of how we'd like   it to respond in different cases and from that 
it learns the sort of the overall personality.   What haven't I asked you that you're thinking 
about a lot that you want people to know? I   feel like we covered a lot of ground. Me, too. But 
I want to know if there's anything on your mind. I don't think so. One of the things that I haven't 
gotten to play with yet, but I'm curious about is   GBT5 being much more in my life, meaning like'


In [8]:
embeddings = HuggingFaceEndpointEmbeddings(
    model="BAAI/bge-m3",
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

In [9]:
vectorstore = FAISS.from_documents(chunks, embeddings)

In [10]:
vectorstore.index_to_docstore_id

{0: 'b5c95118-76d3-4e64-9c5e-5f992a187b17',
 1: '7cc586df-46be-4910-88f7-3b96e09bc32f',
 2: '30b48ad8-86fc-4b70-9b17-a8dc7f4c7f50',
 3: '0718c69d-bb81-42a1-88ab-6c97f3c0b0fe',
 4: 'fab9293b-3bc4-48f1-94e6-4f00f93a8848',
 5: 'a94d4873-f6bc-4bbf-8baf-ae50d8b0ac8d',
 6: 'fccf0200-4d9a-4496-ace6-6577e17d4775',
 7: '28b7d97f-340c-490e-8020-72b1c67eefa1',
 8: 'b69e8cc1-5e8f-48c2-859e-fb8716806a05',
 9: '78f32d51-81e4-402a-9efd-22c85eb59ace',
 10: '0077af3a-64cb-4496-8164-503abb837ada',
 11: '188132a7-b0cc-4f75-adbc-8313cd625c89',
 12: '64d15d59-866f-4b8b-a9b0-a7ce3a9cce95',
 13: '6dde5c58-d2a7-4b71-893c-e42edc443f56',
 14: '8a02c05b-2af9-4e09-88a5-99adfb11905c',
 15: 'bbbfb56c-904c-442c-82e4-2f28fa57b042',
 16: 'dcb48697-1c92-43e3-bd4d-71a8bbf9da81',
 17: '2dc51a57-2847-4ba7-b454-08102ab9c182',
 18: '175aeb07-8c1d-46c0-aa50-1a2377f1698b',
 19: 'af3b53de-690c-429f-b40c-fc6acefca797',
 20: 'c4faf88b-581e-414a-8696-b5e28aedbac6',
 21: '97911f49-89f4-4c5d-8700-2703cf11c977',
 22: 'e565fb76-d5b2-

In [11]:
vectorstore.get_by_ids(['ce0872dc-24b1-41f9-a176-726333a809e6'])

[]

Step2 Retrival

In [12]:
retriver=vectorstore.as_retriever(search_type='similarity', search_kwargs={'k':4})

In [20]:
question="How should society prepare for the rapid changes AI brings?"
retrived_docs=retriver.invoke(question)
retrived_docs

[Document(id='ec239585-a47b-4cac-ab2b-6b2f04f816c2', metadata={}, page_content='get used to them really fast. So we\'ll be like,\xa0\xa0 "Oh, it\'s incredible that this is like being\xa0\nused to cure disease and be like, oh, it\'s\xa0\xa0 extremely scary that models like this are being\xa0\nused to like create new biocurity threats." And\xa0\xa0 then we\'ll also be like, man, it\'s really weird\xa0\nto like live through watching the world speed up\xa0\xa0 so much and you know the economy grows so fast\xa0\nand the like it will feel like vertigo inducing\xa0\xa0 uh the sort of the rate of change and then like\xa0\nhappens with everything else the remarkable\xa0\xa0 ability of of people of humanity to adapt to kind\xa0\nof like any amount of change. we\'ll just be like,\xa0\xa0 "Okay, you know, this is like this is it." Um, a\xa0\nkid born today will never be smarter than AI ever.\xa0\xa0 And a kid born today, by the time that kid like\xa0\nkind of understands the way the world works, w

In [14]:
model = ChatOpenAI(
    model=os.getenv("OPENROUTER_MODEL"),   
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",  # Important: points to OpenRouter
    temperature=0.7,
    # max_tokens=400,
)

In [15]:
prompt=PromptTemplate(
    template='''
You are a helpful assistant.
Answer only from provided transcript context.
If the context is insufficient and your cannot answer question bases on given context only, just say I cannot answer this question 
{context}
Question={question}''',
input_variables=['context','question']

)

In [16]:
def format_docs(retrived_docs):
    context_text='\n\n'.join(doc.page_content for doc in retrived_docs)
    return context_text

In [17]:
parallel_chain=({
    'context':retriver|RunnableLambda(format_docs),
    'question':RunnablePassthrough()
})

In [18]:
parser=StrOutputParser()

In [21]:
main_chain=parallel_chain|prompt|model|parser
result=main_chain.invoke(question)
print(result)

Based on the transcript, society should prepare for AI-driven rapid changes by:  
1. **Emphasizing adaptability**: The transcript highlights that humanity has historically adapted to rapid change, suggesting continuous learning and flexibility will be critical.  
2. **Re-evaluating resource access**: There is speculation that societies may need to discuss how to share AI's benefits equitably, particularly making AI compute "abundant and cheap" to avoid concentration of power or misuse.  
3. **Building on collective infrastructure**: Recognizing society as a "super intelligence" formed by layered contributions (technical, social, political), preparation should involve maintaining and improving shared systems that enable AI advancements.  
4. **Addressing psychological and practical shifts**: People will struggle with the "vertigo" of accelerated change, implying mental readiness and societal norms to process rapid technological evolution.  

The speaker underscores that preparation isn’